In [2]:
#Test Dorna
response: ChatResponse = chat(model='partai/dorna-llama3', messages=[
  {
    'role': 'user',
    'content': 'چرا اینترنت قطع میشه؟',
  },
])
print(response['message']['content'])
print(response.message.content)

این یک مشکل رایج است که اغلب به دلایل مختلف ایجاد می شود. برخی از عللن های متداول قطع اینترنت عبارتند از:

1. اتصال نادرست: ممکن است اتصال شما با ارائه دهنده خدمات اینترنت (ISP) نادرست باشد، در نتیجه قطع دسترسی به اینترنت.
2. خطای شبکه: مشکلاتی مانند خرابی تجهیزات شبکه یا خطاهای مربوطه ممکن است باعث قطع اینترنت شوند.
3. مسدود بودن آدرس IP: اگر آدرس آی پی شما توسط ارائه دهنده خدمات اینترنت مسدود شود، نمی توانید به اینترنت دسترسی داشته باشید.
4. مصرف زیاد باند باس: اگر مصرف زیادی از باند باس در طول یک زمان خاص یا مداوم باشد، ممکن است موجب قطع اینترنت شود.
5. مشکلات تهدید کننده ایمنی: حملات سایبری و ویروس های رایانه ای نیز می توانند باعث قطع اینترنت شوند.

از سوی دیگر، ارائه دهندگان خدمات اینترنت اغلب در نظر می گیرند که به دلیل مسائل فنی یا نگهداری شبکه، اینترنت را برای چند ساعت یا یک روز قطع کنند. در چنین مواردی معمولاً اطلاع رسانی قبلی داده نمی شود و زمانی که مشکل حل شد، اینترنت مجدداً قابل دسترسی است.

در صورت تجربه قطع اینترنت، توصیه می شود که با ارائه دهنده خدمات اینترنت تماس بگیرید 

In [16]:
!pip install faiss-cpu sentence-transformers ollama numpy

     ---------------------------------------- 0.0/44.4 kB ? eta -:--:--
     --------- ------------------------------ 10.2/44.4 kB ? eta -:--:--
     --------- ------------------------------ 10.2/44.4 kB ? eta -:--:--
     -------------------------- ----------- 30.7/44.4 kB 220.2 kB/s eta 0:00:01
     -------------------------- ----------- 30.7/44.4 kB 220.2 kB/s eta 0:00:01
     -------------------------------------- 44.4/44.4 kB 168.2 kB/s eta 0:00:00
   ---------------------------------------- 0.0/13.8 MB ? eta -:--:--
   ---------------------------------------- 0.0/13.8 MB ? eta -:--:--
   ---------------------------------------- 0.0/13.8 MB ? eta -:--:--
   ---------------------------------------- 0.0/13.8 MB ? eta -:--:--
   ---------------------------------------- 0.0/13.8 MB ? eta -:--:--
   ---------------------------------------- 0.1/13.8 MB 563.7 kB/s eta 0:00:25
   ---------------------------------------- 0.1/13.8 MB 563.7 kB/s eta 0:00:25
    ------------------------------

In [4]:
# MANUALLY RAG
from ollama import chat
import numpy as np
import faiss
from sentence_transformers import SentenceTransformer
import os
import re

DOCUMENT_PATHS = [
    r'C:\Users\ASUS\Downloads\Hamrah.txt', #replace path
    r'C:\Users\ASUS\Downloads\vape.txt',
    r'C:\Users\ASUS\Downloads\Shah.txt',
    r'C:\Users\ASUS\Downloads\Khalife.txt',
    r'C:\Users\ASUS\Downloads\carbon.txt',
    r'C:\Users\ASUS\Downloads\takapoo.txt'
]

EMBEDDING_MODEL = 'sentence-transformers/paraphrase-multilingual-mpnet-base-v2'
LLM_MODEL = 'partai/dorna-llama3'
CHUNK_SIZE = 1000
OVERLAP = 200
INDEX_PATH = r'C:\Users\ASUS\Downloads\doc_index.faiss'
CHUNK_MAP_PATH = r'C:\Users\ASUS\Downloads\chunk_map.npy'

class AdvancedRAG:
    def __init__(self):
        self.encoder = SentenceTransformer(EMBEDDING_MODEL)
        self.index = None
        self.chunk_map = []
        
    def create_index(self):
        """Create FAISS index with cosine similarity and document mapping"""
        all_chunks = []
        doc_mapping = []
        
        # Process via CHUNKING (REQ 4 RAG)
        for doc_idx, path in enumerate(DOCUMENT_PATHS):
            with open(path, 'r', encoding='utf-8') as f:
                text = re.sub(r'\s+', ' ', f.read()).strip()
                chunks = [text[i:i+CHUNK_SIZE] for i in range(0, len(text), CHUNK_SIZE - OVERLAP)]
                all_chunks.extend(chunks)
                doc_mapping.extend([doc_idx] * len(chunks))
        
        # Normalized embeddings (REQ 4 cosine similarity)
        embeddings = self.encoder.encode(all_chunks)
        faiss.normalize_L2(embeddings)  
        
        # FAISS index & Mapping
        self.index = faiss.IndexFlatIP(embeddings.shape[1])
        self.index.add(embeddings.astype(np.float32))
        self.chunk_map = np.array(doc_mapping)
        
        # Index  
        faiss.write_index(self.index, INDEX_PATH)
        # Mapping 
        np.save(CHUNK_MAP_PATH, self.chunk_map)
    
    def load_index(self):
        """LOAD EXISTING DATA"""
        self.index = faiss.read_index(INDEX_PATH)
        self.chunk_map = np.load(CHUNK_MAP_PATH, allow_pickle=True)
        
    def query(self, question, doc_index, top_k=5):
        """DOCUMENT-SPECIFIC QUERY WITH COSINE SIMILARITY """
        # Encode 
        query_embed = self.encoder.encode([question])
        # Normalize 
        faiss.normalize_L2(query_embed)
        
        distances, indices = self.index.search(query_embed.astype(np.float32), top_k*3)
    
        relevant_chunks = []
        for idx in indices[0]:
            if self.chunk_map[idx] == doc_index:
                relevant_chunks.append(idx)
            if len(relevant_chunks) >= top_k:
                break
        
        return relevant_chunks

class AnswerGenerator:
    def __init__(self, rag_system):
        self.rag = rag_system
        self.chunks = [] 
        
    def get_answer(self, question, doc_index):
        """GENERATING CONTEXT-AWARE ANSWER"""
        if not self.chunks:
            self._load_chunks()
            
        chunk_indices = self.rag.query(question, doc_index)
        context = "\n".join([self.chunks[idx] for idx in chunk_indices])
        
        prompt = f"""با استفاده از متن زیر به سوال پاسخ دهید:
{context}

اگر پاسخ در متن وجود ندارد عبارت 'پاسخی یافت نشد' را برگردانید

سوال: {question}
پاسخ:"""
        
        response = chat(model=LLM_MODEL, messages=[{'role': 'user', 'content': prompt}])
        return response['message']['content']
    
    def _load_chunks(self):
        """LOAD ALL CHUNKS(LAZY)"""
        self.chunks = []
        for path in DOCUMENT_PATHS:
            with open(path, 'r', encoding='utf-8') as f:
                text = re.sub(r'\s+', ' ', f.read()).strip()
                self.chunks.extend([text[i:i+CHUNK_SIZE] for i in range(0, len(text), CHUNK_SIZE - OVERLAP)])

# MAIN EXE of RAG
if __name__ == "__main__":
    # RAG init
    rag = AdvancedRAG()
    
    if not os.path.exists(INDEX_PATH):
        print("Building optimized index...")
        rag.create_index()
    else:
        print("Loading existing index...")
        rag.load_index()
    # Answer Generator init
    generator = AnswerGenerator(rag)
    
    queries = [
        ("چرا اینترنت همراه اول گوشی وصل نمیشود؟", 0),
        ("چطوری ویپ مورد نظرمو پیدا کنم؟", 1),
        ("شاه عباس که بود؟", 2),
        ("خلیفه سلطان که بود و چه کرد؟", 3),
        ("کربن اکتیو و کربن بلک چه هستند و چه تفاوتی دارند و برای چه استفاده میشن؟", 4),
        ("شرکت تکاپو صنعت نامی چه محصولاتی ارایه میدهد؟ چه چیزی این شرکت را منحصر به فرد میسازد؟ سهام این شرکت صعودی است یا نزولی؟", 5)
    ]
    
    with open(r'C:\Users\ASUS\Downloads\representation.txt', 'w', encoding='utf-8') as f: #replace path
        for q_idx, (query, doc_idx) in enumerate(queries):
            answer = generator.get_answer(query, doc_idx)
            f.write(f"سوال {q_idx+1} ({doc_idx+1}):\n{query}\n\nپاسخ:\n{answer}\n\n{'='*50}\n\n")
            print(f"پردازش سوال {q_idx+1}/{len(queries)} تکمیل شد")

print("تمامی سوالات با موفقیت پردازش شدند!")

Building optimized index...
پردازش سوال 1/6 تکمیل شد
پردازش سوال 2/6 تکمیل شد
پردازش سوال 3/6 تکمیل شد
پردازش سوال 4/6 تکمیل شد
پردازش سوال 5/6 تکمیل شد
پردازش سوال 6/6 تکمیل شد
تمامی سوالات با موفقیت پردازش شدند!


In [1]:
!pip install langchain chromadb sentence-transformers ollama

     ---------------------------------------- 0.0/67.3 kB ? eta -:--:--
     ---------------------------------------- 0.0/67.3 kB ? eta -:--:--
     ---------------------------------------- 0.0/67.3 kB ? eta -:--:--
     ------------------ --------------------- 30.7/67.3 kB ? eta -:--:--
     ------------------ --------------------- 30.7/67.3 kB ? eta -:--:--
     -------------------------------------- 67.3/67.3 kB 523.1 kB/s eta 0:00:00
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
     ---------------------------------------- 0.0/42.9 kB ? eta -:--:--
     ---------------------------------------- 0.0/42.9 kB ? eta -:--:--
     ---------------------------- ----------- 30.7/42.9 kB 1.3 MB/s eta 0:00:01
    

  You can safely remove it manually.
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
streamlit 1.30.0 requires packaging<24,>=16.8, but you have packaging 24.2 which is incompatible.
streamlit 1.30.0 requires protobuf<5,>=3.20, but you have protobuf 5.29.3 which is incompatible.
streamlit 1.30.0 requires tenacity<9,>=8.1.0, but you have tenacity 9.0.0 which is incompatible.


In [3]:
!pip install -U langchain-community

   ---------------------------------------- 0.0/2.5 MB ? eta -:--:--
   ---------------------------------------- 0.0/2.5 MB ? eta -:--:--
   ---------------------------------------- 0.0/2.5 MB ? eta -:--:--
   ---------------------------------------- 0.0/2.5 MB ? eta -:--:--
   ---------------------------------------- 0.0/2.5 MB ? eta -:--:--
   - -------------------------------------- 0.1/2.5 MB 651.6 kB/s eta 0:00:04
   - -------------------------------------- 0.1/2.5 MB 651.6 kB/s eta 0:00:04
   -- ------------------------------------- 0.1/2.5 MB 561.1 kB/s eta 0:00:05
   --- ------------------------------------ 0.2/2.5 MB 827.9 kB/s eta 0:00:03
   ----- ---------------------------------- 0.3/2.5 MB 933.7 kB/s eta 0:00:03
   ------ --------------------------------- 0.4/2.5 MB 1.1 MB/s eta 0:00:02
   -------- ------------------------------- 0.6/2.5 MB 1.4 MB/s eta 0:00:02
   ----------- ---------------------------- 0.7/2.5 MB 1.5 MB/s eta 0:00:02
   ----------- ----------------------

In [1]:
# CHROMOA
from langchain.vectorstores import Chroma
from langchain.embeddings import SentenceTransformerEmbeddings
from langchain.schema import Document
from ollama import chat
import os
import re

DOCUMENT_PATHS = [
    r'C:\Users\ASUS\Downloads\Hamrah.txt', #replace path
    r'C:\Users\ASUS\Downloads\vape.txt',
    r'C:\Users\ASUS\Downloads\Shah.txt',
    r'C:\Users\ASUS\Downloads\Khalife.txt',
    r'C:\Users\ASUS\Downloads\carbon.txt',
    r'C:\Users\ASUS\Downloads\takapoo.txt'
]

EMBEDDING_MODEL = 'sentence-transformers/paraphrase-multilingual-mpnet-base-v2'
LLM_MODEL = 'partai/dorna-llama3'
CHUNK_SIZE = 1000
OVERLAP = 200
CHROMA_PERSIST_DIR = r'C:\Users\ASUS\Downloads\chroma_db'  

class ChromaRAGSystem:
    def __init__(self):
        # Init embedding model
        self.embeddings = SentenceTransformerEmbeddings(model_name=EMBEDDING_MODEL)
        # Vector store instance
        self.vector_db = None
        
    def build_vector_store(self):
        """Process documents and create Chroma vector store"""
        all_docs = []
        

        for doc_idx, path in enumerate(DOCUMENT_PATHS):
            with open(path, 'r', encoding='utf-8') as f:
                text = re.sub(r'\s+', ' ', f.read()).strip()
                # sliding window chunking
                chunks = [
                    text[i:i+CHUNK_SIZE] 
                    for i in range(0, len(text), CHUNK_SIZE - OVERLAP)
                ]
                # LangChain documents with metadata
                for chunk in chunks:
                    all_docs.append(Document(
                        page_content=chunk,
                        metadata={"source_doc": doc_idx}
                    ))
        
        # Chroma vector store
        self.vector_db = Chroma.from_documents(
            documents=all_docs,
            embedding=self.embeddings,
            persist_directory=CHROMA_PERSIST_DIR
        )
        self.vector_db.persist()
        
    def load_vector_store(self):
        """Load existing Chroma vector store"""
        self.vector_db = Chroma(
            persist_directory=CHROMA_PERSIST_DIR,
            embedding_function=self.embeddings
        )
        
    def document_query(self, query, doc_index, top_k=5):
        """Retrieve context from specific document"""
        # Chroma metadata filtering
        results = self.vector_db.similarity_search(
            query=query,
            k=top_k,
            filter={"source_doc": doc_index}
        )
        return [doc.page_content for doc in results]

class AnswerGenerator:
    def __init__(self, rag_system):
        self.rag = rag_system
        
    def generate_response(self, question, doc_index):
        """Generate context-aware answer using LLM"""
        # Retrieve relevant context
        context_chunks = self.rag.document_query(question, doc_index)
        context = "\n".join(context_chunks)
        
        prompt = f"""با استفاده از متن زیر به سوال پاسخ دهید:
{context}

اگر پاسخ در متن وجود ندارد عبارت 'پاسخی یافت نشد' را برگردانید

سوال: {question}
پاسخ:"""
        
        response = chat(model=LLM_MODEL, messages=[{'role': 'user', 'content': prompt}])
        return response['message']['content']

if __name__ == "__main__":
    rag_system = ChromaRAGSystem()
    
    # Init vector store
    if not os.path.exists(CHROMA_PERSIST_DIR):
        print("Creating new vector store...")
        rag_system.build_vector_store()
    else:
        print("Loading existing vector store...")
        rag_system.load_vector_store()
    
    # Init answer generator
    answer_engine = AnswerGenerator(rag_system)

    queries = [
        ("چرا اینترنت همراه اول گوشی وصل نمیشود؟", 0),
        ("چطوری ویپ مورد نظرمو پیدا کنم؟", 1),
        ("شاه عباس که بود؟", 2),
        ("خلیفه سلطان که بود و چه کرد؟", 3),
        ("کربن اکتیو و کربن بلک چه هستند و چه تفاوتی دارند و برای چه استفاده میشن؟", 4),
        ("شرکت تکاپو صنعت نامی چه محصولاتی ارایه میدهد؟ چه چیزی این شرکت را منحصر به فرد میسازد؟ سهام این شرکت صعودی است یا نزولی؟", 5)
    ]
    
    with open(r'C:\Users\ASUS\Downloads\representation2.txt', 'w', encoding='utf-8') as output_file: #repalce path
        for q_num, (query, doc_idx) in enumerate(queries):
            answer = answer_engine.generate_response(query, doc_idx)
            output_file.write(f"سوال {q_num+1} ({doc_idx+1}):\n{query}\n\nپاسخ:\n{answer}\n\n{'='*50}\n\n")
            print(f"پردازش سوال {q_num+1}/{len(queries)} تکمیل شد")

print("تمامی سوالات با موفقیت پردازش شدند!")

C:\Users\ASUS\AppData\Local\Temp\ipykernel_1004\3712172232.py:26: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  self.embeddings = SentenceTransformerEmbeddings(model_name=EMBEDDING_MODEL)


Loading existing vector store...


C:\Users\ASUS\AppData\Local\Temp\ipykernel_1004\3712172232.py:60: LangChainDeprecationWarning: The class `Chroma` was deprecated in LangChain 0.2.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-chroma package and should be used instead. To use it run `pip install -U :class:`~langchain-chroma` and import as `from :class:`~langchain_chroma import Chroma``.
  self.vector_db = Chroma(


پردازش سوال 1/6 تکمیل شد
پردازش سوال 2/6 تکمیل شد
پردازش سوال 3/6 تکمیل شد
پردازش سوال 4/6 تکمیل شد
پردازش سوال 5/6 تکمیل شد
پردازش سوال 6/6 تکمیل شد
تمامی سوالات با موفقیت پردازش شدند!
